In [7]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

In [8]:
# Read primers
filename_primers = '2019nCoV_primers.fa'
primer_list = read_fasta(filename_primers)

#make the new dictionaries, primer and types
primer = dict()
types = dict()

#sort the primer according to their set
for p_h, p_s in primer_list.items():   
    
    #apply revcomp function to reverse primer for genome sequnece comparison
    if p_h[-1] == 'R':
        p_s = revcomp(p_s)
   
    #primer.keys have primer set name / primer.values have types which is another dictionary
    #types.keys have 3 type of primer, forward(F), reverse(R), prove(P) / types.values have primer sequences
    if p_h[:-2] in primer.keys():  
        primer[p_h[:-2]][p_h[-1]] = p_s       
    else:
        types[p_h[-1]] = p_s
        primer[p_h[:-2]] = types
        types = dict()
        
print(primer)
print("there are", len(primer),"sets of primer")

{'2019-nCoV_N1': {'F': 'GACCCCAAAATCAGCGAAAT', 'R': 'CAGATTCAACTGGCAGTAACCAGA', 'P': 'ACCCCGCATTACGTTTGGTGGACC'}, '2019-nCoV_N2': {'F': 'TTACAAACATTGGCCGCAAA', 'R': 'TTCTTCGGAATGTCGCGC', 'P': 'ACAATTTGCCCCCAGCGCTTCAG'}, '2019-nCoV_N3': {'F': 'GGGAGCCTTGAATACACCAAAA', 'R': 'CAATGCTGCAATCGTGCTACA', 'P': 'AYCACATTGGCACCCGCAATCCTG'}, 'RP': {'F': 'AGATTTGGACCTGCGAGCG', 'R': 'ACTTGTGGAGACAGCCGCTC', 'P': 'TTCTGACCTGAAGGCTCTGCGCG'}}
there are 4 sets of primer


In [9]:
# Q1. What is the length of amplicons generated by primers? Are they all same among the genomes?

# Read genomes
filename_genomes = '2019nCoV_genomes.2020_03_27.fa'
genome_list = read_fasta(filename_genomes)

A = dict()
count = dict()
SUM = 0

# p_set: string, <2019-nCoV_N1,2019-nCoV_N2,2019-nCoV_N3, RP>
# p_type: dictionary, <F:seqeunces, R:sequences. P:sequences>
print("Q1: What is the length of amplicons generated by primers? Are they all same among the genomes?")
for p_set, p_type in primer.items():
    for g_h, g_seq in genome_list.items():
        
        #p_type["F"]: forward primer sequence, p_type["R"]: reverse primer sequnece
        if p_type["F"] in g_seq and p_type["R"] in g_seq:
            # we have to add the reverse primer sequence length, because amplicon include both forward and reverse primer
            amplicon_len = abs(g_seq.find(p_type["R"])-g_seq.find(p_type["F"]))+len(p_type["R"])
            A[g_h] = amplicon_len            
    
    #count the overlapping amplicon lengths and 
    for c in A.values():
        SUM = SUM + c
        if c not in count.keys():
            count[c] = 1
        else:
            count[c] += 1
    
    #print the results
    if len(count) == 1:
        print("\n in", p_set,", mean of amplicon lengths is", SUM/len(A))
        print("** length of amplicons are all same among the genomes in", p_set,"primer sets")
    elif len(count) == 0:
        print("\n no amplicons are detected by",p_set, "primer sets with these genome sets")
    else:
        print("in", p_set,", mean of amplicon lengths is", SUM/len(A))
        print("\n amplicon size are various among the genomes")
    
    #reset the SUM, A, count when start with new primer sets
    SUM = 0
    A = dict() 
    count = dict()

Q1: What is the length of amplicons generated by primers? Are they all same among the genomes?

 in 2019-nCoV_N1 , mean of amplicon lengths is 72.0
** length of amplicons are all same among the genomes in 2019-nCoV_N1 primer sets

 in 2019-nCoV_N2 , mean of amplicon lengths is 67.0
** length of amplicons are all same among the genomes in 2019-nCoV_N2 primer sets

 in 2019-nCoV_N3 , mean of amplicon lengths is 72.0
** length of amplicons are all same among the genomes in 2019-nCoV_N3 primer sets

 no amplicons are detected by RP primer sets with these genome sets


In [10]:
# Q2. Can these primers detect all SARS-CoV-2 genomes?
# '../data/2019nCoV_genomes.2020_02_03.fa' and '../data/2019nCoV_genomes.2020_03_27.fa'

# Read genomes of SARS-CoV-2 genomes
filename_genomes = '2019nCoV_genomes.2020_02_03.fa'
genome_list_old = read_fasta(filename_genomes)

# add 2019_nCoV_genomes.2020_02_03.fa genome list to my genome list dictionary
genome_list.update(genome_list_old)

count = 0
# p_set: string, <2019-nCoV_N1,2019-nCoV_N2,2019-nCoV_N3, RP>
# p_type: dictionary, <F:seqeunces, R:sequences. P:sequences>
print("Q2. Can these primers detect all SARS-CoV-2 genomes?")
for p_set, p_type in primer.items():
    for g_h, g_seq in genome_list.items():
        
        #p_type["F"]: forward primer sequence, p_type["R"]: reverse primer sequnece
        if p_type["F"] in g_seq and p_type["R"] in g_seq:
            count += 1            
    
    #print the results
    if len(genome_list) == count:
        print("\n",p_set,"primer can detect all SARS-CoV-2 genomes")
    elif count == 0:
        print("\n",p_set,"primer can't detect any SARS-CoV-2 genomes")
    else:
        print("\n",p_set,"primer can't detect all SARS-CoV-2 genomes, but can detect",count,"types of SARS-CoV-2 genomes")
        
    
    #reset count when start with new primer sets
    count = 0

Q2. Can these primers detect all SARS-CoV-2 genomes?

 2019-nCoV_N1 primer can detect all SARS-CoV-2 genomes

 2019-nCoV_N2 primer can detect all SARS-CoV-2 genomes

 2019-nCoV_N3 primer can detect all SARS-CoV-2 genomes

 RP primer can't detect any SARS-CoV-2 genomes


In [11]:
# Q3. Can these primers detect MERS genomes? How about SARS genomes? (you can find those genomes under '../data' directory).

# Read genomes
MERS = 'MERS_genomes.2020_02_03.fa'
SARS = 'SARS_genomes.2020_02_03.fa'
MERS_list = read_fasta(MERS)
SARS_list = read_fasta(SARS)
S_count = 0
M_count = 0

# p_set: string, <2019-nCoV_N1,2019-nCoV_N2,2019-nCoV_N3, RP>
# p_type: dictionary, <F:seqeunces, R:sequences. P:sequences>
print("Q3. Can these primers detect MERS genomes? How about SARS genomes?")
for p_set, p_type in primer.items():
    
    for g_h, g_seq in MERS_list.items():
        
        #p_type["F"]: forward primer sequence, p_type["R"]: reverse primer sequnece
        if p_type["F"] in g_seq and p_type["R"] in g_seq:
            M_count += 1            
    
    for g_h, g_seq in SARS_list.items():
        
        #p_type["F"]: forward primer sequence, p_type["R"]: reverse primer sequnece
        if p_type["F"] in g_seq and p_type["R"] in g_seq:
            S_count += 1            

    #print the results
    if M_count == 0 and S_count == 0:
        print("\n",p_set,"primer set can't detect both MERS and SARS genome")  
    elif M_count != 0:
        print("\n",p_set, "primer set can detect MERS genome")
    elif S_count != 0:
        print("\n",p_set, "primer set can detect SARS genome")
    else:
        print("\n",p_set, "primer set can detect both MERS and SARS genomes")
    
    #reset S_count and M_count when start with new primer set
    S_count = 0
    M_count = 0

Q3. Can these primers detect MERS genomes? How about SARS genomes?

 2019-nCoV_N1 primer set can't detect both MERS and SARS genome

 2019-nCoV_N2 primer set can't detect both MERS and SARS genome

 2019-nCoV_N3 primer set can't detect both MERS and SARS genome

 RP primer set can't detect both MERS and SARS genome


In [12]:
# I copied Hyerinkim's work tried to understand whats going on
# this work doesnt belong to me
#sorry fot that((